In [27]:
# Dependencies
from bs4 import BeautifulSoup
import requests
from splinter import Browser
import pymongo
import pandas as pd

In [28]:
# Set URL's
news_url= "https://mars.nasa.gov/news/"
img_url= "https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars"
fact_url= "https://space-facts.com/mars/"
usgs_url= "https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars"

In [87]:
executable_path = {'executable_path': 'chromedriver.exe'}
browser = Browser('chrome', **executable_path, headless=False)

In [42]:
# Retrieve page with the browser module
browser.visit(news_url)

html = browser.html

soup = BeautifulSoup(html, 'html.parser')

In [43]:
# Create a variable to store the scraped article title and paragraph description
header = soup.find('div', class_='list_text')
news_title = header.find('div', class_='content_title').text
news_p = header.find('div', class_='article_teaser_body').text
print(news_title)
print(news_p)

Testing Proves Its Worth With Successful Mars Parachute Deployment
The giant canopy that helped land Perseverance on Mars was tested here on Earth at NASA’s Wallops Flight Facility in Virginia.


In [147]:
# Setting up new Image URL as the link to the NASA jpl images library doesn't have a featured image anymore
# Retrieve page with the browser module
img_url2 = "https://mars.nasa.gov/"
browser.visit(img_url2)

# Get browser info into beautiful soup
html = browser.html
soup = BeautifulSoup(html, 'html.parser')

# image = soup.find('div', class_='window')
image = soup.find('div', class_='window').a['href']
browser.visit(image)

html = browser.html
soup = BeautifulSoup(html, 'html.parser')

image_url = soup.find('figure', class_='lede').a['href']
featured_image_url = f'https://mars.nasa.gov{image_url}'
print(featured_image_url)

https://mars.nasa.gov/system/resources/detail_files/25538_PIA24310_-_Farley_Traverse_-_unannotated-web.jpg


In [148]:
# Old Code that doesn't go to the correct link as the website has changed as of 02/26/2021 

# # Retrieve page with the browser module
# browser.visit(img_url)

# # Click the Full Image link
# browser.click_link_by_partial_text("FULL IMAGE")

# # Click the more info link
# browser.click_link_by_partial_text("more info")

# # Get browser info into beautiful soup
# html = browser.html
# soup = BeautifulSoup(html, 'html.parser')

# image_url = soup.find('figure', class_='lede').a['href']
# featured_image_url = f'https://www.jpl.nasa.gov{image_url}'
# print(featured_image_url)

In [149]:
# Use pandas to scrape the table containing Mars facts
facts_tbl = pd.read_html(fact_url)
facts_tbl

[                      0                              1
 0  Equatorial Diameter:                       6,792 km
 1       Polar Diameter:                       6,752 km
 2                 Mass:  6.39 × 10^23 kg (0.11 Earths)
 3                Moons:            2 (Phobos & Deimos)
 4       Orbit Distance:       227,943,824 km (1.38 AU)
 5         Orbit Period:           687 days (1.9 years)
 6  Surface Temperature:                   -87 to -5 °C
 7         First Record:              2nd millennium BC
 8          Recorded By:           Egyptian astronomers,
   Mars - Earth Comparison             Mars            Earth
 0               Diameter:         6,779 km        12,742 km
 1                   Mass:  6.39 × 10^23 kg  5.97 × 10^24 kg
 2                  Moons:                2                1
 3      Distance from Sun:   227,943,824 km   149,598,262 km
 4         Length of Year:   687 Earth days      365.24 days
 5            Temperature:     -87 to -5 °C      -88 to 58°C,
           

In [150]:
# Create a data frame from the first table
mars_df = facts_tbl[0]

# Add Column names
mars_df.columns=['Description','Mars']

# Set the index to the description
mars_df.set_index('Description', inplace=True)
mars_df.head()

,Mars
Description,
Equatorial Diameter:,"6,792 km"
Polar Diameter:,"6,752 km"
Mass:,6.39 × 10^23 kg (0.11 Earths)
Moons:,2 (Phobos & Deimos)
Orbit Distance:,"227,943,824 km (1.38 AU)"


In [151]:
# Use pandas to convert to a HTML table
mars_facts = mars_df.to_html()

In [152]:
# Retrieve page with the browser module
browser.visit(usgs_url)

html = browser.html

soup = BeautifulSoup(html, 'html.parser')

In [153]:
hemisphere_image_url = []

urls = soup.find('div', class_='collapsible results')
hemispheres = urls.find_all('div', class_='item')

for hemisphere in hemispheres:
    title =  hemisphere.find('h3').text
    title = title.replace(' Enhanced', '')
    
    link = 'https://astrogeology.usgs.gov/' + hemisphere.find('a')['href']
    
    browser.visit(link)
    html = browser.html
    img_soup = BeautifulSoup(html, 'html.parser')
    
    img_url = img_soup.find('div', class_='downloads').a['href']

    hemisphere_image_url.append({'title': title, 'img_url': img_url})

hemisphere_image_url

[{'title': 'Cerberus Hemisphere',
  'img_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/cerberus_enhanced.tif/full.jpg'},
 {'title': 'Schiaparelli Hemisphere',
  'img_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/schiaparelli_enhanced.tif/full.jpg'},
 {'title': 'Syrtis Major Hemisphere',
  'img_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/syrtis_major_enhanced.tif/full.jpg'},
 {'title': 'Valles Marineris Hemisphere',
  'img_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/valles_marineris_enhanced.tif/full.jpg'}]